In [1]:
import numpy as np 
import mne
import os
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils as np_utils
from sklearn.model_selection import train_test_split
from EEGModels import EEGNet, DeepConvNet
from htnet_model import htnet

In [2]:
tf.config.list_physical_devices(device_type='GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [27]:
path=os.path.abspath(os.path.join(os.path.dirname(" __file__" ), '../..', 'bci3_npz_raw'))
data=np.load(path+"/"+"bci_3.npz")
X=data['X']
events_ts=data['events'].squeeze()
event_id=dict(finger=-1, tongue=1)
n_channels=X.shape[1]
s_freq=1000

In [28]:
info=mne.create_info(n_channels,sfreq=s_freq)
print(info)

<Info | 7 non-empty values
 bads: []
 ch_names: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, ...
 chs: 64 MISC
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 64
 projs: []
 sfreq: 1000.0 Hz
>


In [5]:
events_ts_segmented=np.empty((events_ts.shape[0]*3),dtype=int)
j=0
for i in range(events_ts.shape[0]):
    events_ts_segmented[j]=events_ts[i]
    events_ts_segmented[j+1]=events_ts[i]
    events_ts_segmented[j+2]=events_ts[i]
    j=j+3

In [6]:
len=events_ts_segmented.shape[0]
ev=[i*s_freq*1 for i in range(len)]

In [7]:
events=np.column_stack((np.array(ev),np.zeros(len, dtype=int),np.array(events_ts_segmented)))
events

array([[     0,      0,     -1],
       [  1000,      0,     -1],
       [  2000,      0,     -1],
       ...,
       [831000,      0,     -1],
       [832000,      0,     -1],
       [833000,      0,     -1]])

In [8]:
j=0
X_segmented=np.empty((X.shape[0]*3,64,1000))
for i in range(X.shape[0]):
    X_segmented[j,:,:]=X[i,:,:1000]
    X_segmented[j+1,:,:]=X[i,:,1000:2000]
    X_segmented[j+2,:,:]=X[i,:,2000:3000]
    j=j+3

In [9]:
X_segmented.shape, events.shape

((834, 64, 1000), (834, 3))

In [10]:
tmin=0
epochs=mne.EpochsArray(X_segmented,info,events,tmin,event_id)

In [20]:
X=epochs.get_data()
y=epochs.events[:, -1]

In [13]:
test_data=data['X_test']
true_labels=np.loadtxt('true_labels_bci3.txt')

In [14]:
j=0
test_segmented=np.empty((test_data.shape[0]*3,64,1000))
for i in range(test_data.shape[0]):
    test_segmented[j,:,:]=test_data[i,:,:1000]
    test_segmented[j+1,:,:]=test_data[i,:,1000:2000]
    test_segmented[j+2,:,:]=test_data[i,:,2000:3000]
    j=j+3

In [18]:
true_labels_segmented=np.empty((true_labels.shape[0]*3),dtype=int)
j=0
for i in range(true_labels.shape[0]):
    true_labels_segmented[j]=true_labels[i]
    true_labels_segmented[j+1]=true_labels[i]
    true_labels_segmented[j+2]=true_labels[i]
    j=j+3

In [15]:
np.savez_compressed("bci3epochs.npz",X=X,y=y,X_test=test_segmented,y_test=true_labels_segmented)